# Charging Experiments

## Imports

In [ ]:
from pathlib import Path
from datetime import date
import os
import subprocess
from datetime import datetime
from copy import deepcopy
from revcan.reverse_engineering.models.experiment import Experiment

## Create Necessary Experiment Files

### TODO: 
Set & check the following parameters:
- **Car Model File Name**: Specify the name of the car model file (`{model}_{vin}_{date-created}.json`).
- **Car Model Directory**: Check the directory path where the car model file is stored (Default: `../../data/car_metadata`).
- **Experiment Folder**: Set the path for the experiment folder where the results will be saved (Default: `../../data/experiments/{car}/{experiment}/{date-today}`).

In [ ]:
car_model_file_name = "model_vin_date-created.json"
car_model_dir = "../../data/car_metadata"
experiment_folder = "../../data/experiments/car/charging/"

car_model_file = f"{car_model_dir}/{car_model_file_name}"
car_model_file_path = os.path.abspath(car_model_file)
Path(experiment_folder).mkdir(parents=True, exist_ok=True)

# Generate new folder for experiments
timestamp = datetime.now().strftime("%Y_%m_%d_%H-%M-%S")
experiment_folder = os.path.join(experiment_folder, timestamp)
Path(experiment_folder).mkdir(parents=True, exist_ok=True)

duty_cycles=["11","21","31","41","51"]


for duty_cycle in duty_cycles:

    experiment_file_path = os.path.join(experiment_folder, duty_cycle+".json")
    experiment_name = f"Charging with simplebox PWM Duty Cylce {duty_cycle}%"
    experiment_description = "Charging parameter discovery"
    signal_selection ="all"
    print(f"Experiment file path: {experiment_file_path}")
    !python ../scripts_for_doip_new/04_create_experiment.py --car_model_file_path {car_model_file_path} --experiment_file_path '{experiment_file_path}' --experiment_name '{experiment_name}' --experiment_description '{experiment_description}' --signal_selection {signal_selection}


## [Optional]: Set Experiment Names manually

In [ ]:
car_model_file_name = "model_vin_date-created.json"
car_model_dir = "../../data/car_metadata"
experiment_folder = "../../data/experiments/car/charging/2025_03_20_11-45-22"
duty_cycles=["11","21","31","41","51"]


car_model_file = f"{car_model_dir}/{car_model_file_name}"
car_model_file_path = os.path.abspath(car_model_file)


## Measurement - AC Charging
**Before** executing the next Cell:

- Set duty cycle of simple box to 11% which is equal to ~4.2kW


Start measurement


In order to apply **pre-filtering** strategy:

After **2** measurements: 

- execute the Filter Cells below and **return**
- complete the remaining measurements.
- Continue normal flow of cells.



In [ ]:
#--experiment_file_path /home/alex/Documents/idp/can_hacking/data/experiments/VW_ID.3_2020_WVWZZZE1ZMP014405_all-dids_2024-11-22.json --activate_logging true
!python ../scripts_for_doip_new/05_read_data.py --experiment_file_path '{os.path.join(experiment_folder, "11"+".json")}' --activate_logging true --num_samples 5


Before executing next Cell:

Set duty cycle of simple box to 21% which is equal to ~6.2kW

In [ ]:
!python ../scripts_for_doip_new/05_read_data.py --experiment_file_path '{os.path.join(experiment_folder, "21"+".json")}' --activate_logging true --num_samples 5

Before executing next Cell:

Set duty cycle of simple box to 31% which is equal to ~8.4kW

In [ ]:
!python ../scripts_for_doip_new/05_read_data.py --experiment_file_path '{os.path.join(experiment_folder, "31"+".json")}' --activate_logging true --num_samples 5

Before executing next Cell:

Set duty cycle of simple box to 41% which is equal to ~10.5kW

In [ ]:
!python ../scripts_for_doip_new/05_read_data.py --experiment_file_path '{os.path.join(experiment_folder, "41"+".json")}' --activate_logging true --num_samples 5

# Evaluation


## [Optional] Display Experiment Metadata

In [ ]:
# Display AC Experiment
for duty_cycle in duty_cycles:
    experiment_file = os.path.join(experiment_folder, f"accelerator_pedal_position_{duty_cycle}.json")
    !python ../scripts_for_doip_new/display_experiment_metadata.py --experiment_file_path "{experiment_file}"


## Load Experiments

Load all relevant experiments needed for this analysis

In [ ]:
experiment_files = {}
experiments = {}

for duty_cycle in duty_cycles:
    experiment_files[duty_cycle] = os.path.join(experiment_folder, f"{duty_cycle}.json")
    experiments[duty_cycle] = Experiment.load(experiment_files[duty_cycle])
pass



## Add Ground truth
Add ground truth data for charging experiment. By experiment design these are fixed values, and are added manually in this case

In [ ]:
from datetime import timedelta
import datetime
from typing import List, Dict
from copy import deepcopy
from revcan.reverse_engineering.models.experiment import Extern_Signal, Value, Experiment
import numpy as np

duty_cycles_map={"11":4200,"21":6200,"31":8400,"41":10500,"51":10500}

#pwm = 3.077 ∗ I − 8.462
# I = (pwm+8.462)/3.077

for duty_cycle in duty_cycles:
    experiments[duty_cycle].external_measurements.append(Extern_Signal( name = "Charging_Power", id = 1, values = []))
    values:List[Value] = []
    for value in experiments[duty_cycle].measurements[0]:
        values.append(Value(value= [duty_cycles_map[duty_cycle]], time= value.time))
    experiments[duty_cycle].external_measurements[0].values = deepcopy(values)
    print(f"Experiment duty cycle {duty_cycle} : {len(experiments[duty_cycle].external_measurements[0].values)} ground truth values with value {experiments[duty_cycle].external_measurements[0].values[0].value}")





Concat all experiments

In [ ]:
combined_experiment = deepcopy(experiments[duty_cycles[0]])
combined_experiment.name = f"Non constant singals for charging experiment"
combined_experiment.description = f"Non constant signals for charging experiment with {len(duty_cycles)} different duty cycles."

for duty_cylce in duty_cycles[1:]:
    combined_experiment.experiment_runtime_seconds += experiments[duty_cylce].experiment_runtime_seconds
    for measurement in experiments[duty_cylce].measurements:
        for x in combined_experiment.measurements:
            if x.serverid == measurement.serverid:
                if x.did == measurement.did:
                    x.values.extend(measurement.values)
                    break


Keep only non constant signals

In [ ]:

number_measurements = len(combined_experiment.measurements)
print(f"Experiment contains {number_measurements} measurements before filtering", )
print("Applaying remove constant values filter")

combined_experiment.keep_non_constant_signals(combined_experiment)

number_measurements = len(combined_experiment.measurements)
print(f"Experiment contains {number_measurements} measurements after filtering", )

Remove all previously found constant signals from individual experiments

In [ ]:
for duty_cycle in duty_cycles:
    experiments[duty_cycle].keep_signals_by_list(signals_list=combined_experiment.measurements)
    experiment_file = os.path.join(experiment_folder, f"accelerator_pedal_position_{duty_cycle}.json")
    experiments[duty_cycle].save(f"{experiment_file}")
    !python ../scripts_for_doip_new/display_experiment_metadata.py --experiment_file_path "{experiment_file}"

Save combined experiment file

In [ ]:
combined_experiment_file = os.path.join(experiment_folder, f"combined.json")
combined_experiment.save(f"{combined_experiment_file}")
!python ../scripts_for_doip_new/display_experiment_metadata.py --experiment_file_path "{combined_experiment_file}"

### End of Experiment 

Continue with Linear evaluation